In [ ]:
!pip install geocoder
!pip install lxml html5lib
!pip install tqdm

In [2]:
import requests
import pandas as pd
import numpy as np
import geocoder
import re

In [3]:
from tqdm import tnrange, tqdm_notebook
from tqdm import tqdm
tqdm.pandas(desc="p")


In [4]:
# cache place queries  latlng 
geo_cache = {}

In [ ]:
# take a string, get a latlng, with this simple cache
def get_lat_lng(x):
    debug = False
#   tqdm.write("Done task %i" % i)
    if debug: print('\n'+x)
    if x in geo_cache:
        res = geo_cache[x]
        return res
    g = geocoder.google(x)
    if g.latlng:
        geo_cache[x] = g.latlng
        res = g.latlng
    else:
        res = [pd.np.nan, pd.np.nan]
    if debug: print(res)
    return res

In [133]:
print(get_lat_lng('Brownstone Bagel, Brooklyn, NY'))

[nan, nan]


### OSHA Data Part 1: XML of recent fatalities and catastrophe
The recent stuff is in xml, the older stuff is csv, don't worry, they don't have the same fields either

In [6]:
# https://stackoverflow.com/a/10077069/83859
from xml.etree import ElementTree
from collections import defaultdict

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [7]:
response = requests.get('https://www.osha.gov/dep/fatcat/fatcats.xml')
# print(response)

tree = ElementTree.fromstring(response.content)

In [8]:
pd.set_option('max_colwidth', 800)


In [10]:
# OK STARTING HERE AGAIN

# this has 2016 and 2017, in xml
osha_current = pd.DataFrame.from_records( etree_to_dict(tree)['events']['incident'] )
# print(osha_current[:4].employer[3])
# osha_current[:1]

# just ny state for (mostly)
# osha_current = osha_current[osha_current.employer.str.contains('NY')]
# drop what we're not using

In [ ]:
# osha_current.drop(['fatcat', 'inspection', 'victim'], axis=1, inplace=True)

In [30]:
osha_current[-4:]

,date,description,employer,fatcat,inspection,victim
569,2016-06-27,Worker died from hantavirus after being exposed to mouse droppings while cleaning cabins.,"Marzano Mountain Retreat, Torreon, NM 87061",Fatality,1154167,Shawnee Romero
570,2016-05-24,Worker struck and killed by falling tree.,"Raymond Trent dba Raymond Trent, Morganfield, KY 42437",Fatality,1186003.015,James Finwick
571,None,Worker died after being struck by forklift.,"Genuine Parts Company, 30-03 Review Ave LONG ISLAND CITY NY 11101",Fatality,1209887,None
572,None,Worker fatally crushed between tow truck and another vehicle.,Berkebile Auto Service 10 D Street JOHNSTOWN PA 15906,Fatality,1197414,James Harr


In [63]:
t1 = "Berkebile Auto Service 10 D Street JOHNSTOWN PA 15906" # no commas 
t2 ="Marzano Mountain Retreat, Torreon, NM 87061"
t3 = "Genuine Parts Company, 30-03 Review Ave LONG ISLAND CITY NY 11101"

# ','.join(t1.split(',')[1:]).strip() # no dice
# ','.join(t2.split(',')[1:]).strip()
# ','.join(t3.split(',')[1:]).strip()

# get_lat_lng(t1) # has 
# get_lat_lng(t2) # has
get_lat_lng(t2) # lacks
# get_lat_lng(','.join(t3.split(',')[1:]).strip()) # has, lacks without dropping the business name

# osha_current.apply(lambda row: row.employer.split(',')[-1].strip(), axis=1)

# osha_current.apply(lambda row: row.employer, axis=1)

[34.6875559, -106.3515895]

In [ ]:

def get_osha_current_loc(row): 
    place = row.employer
    # try it with no processing first
    loc = get_lat_lng(place)
    if loc[0] is not pd.np.NaN:
        return loc
    # then try it dropping a possible business name, or whatever is before the first comma
    if place.find(',') is not -1:
        no_b_name = ','.join(place.split(',')[1:]).strip()
        loc = get_lat_lng(no_b_name)
        if loc[0] is not pd.np.NaN:
            return loc        
    m = re.search("\d", place)
    # now drop everything before the first digit and try that
    if m:
        after_digit = place[m.start():]
        loc = get_lat_lng(after_digit)
        if loc[0] is not pd.np.NaN:
            return loc  
    # this should be NaN, NaN if we failed to find something, from the first one at least
    return loc 


    # this is the one from the ny map, needs to be broader now
    # just try it straight?
    # actually XXX for deathmap, should use the [1:] method!
#     return get_lat_lng(row.employer.split(',')[-1].strip())
#     "MHP Real Estate Services, 180 Maiden Lane NEW YO".split(',')[-1].strip()
# osha_current[lat]

# get_osha_current_loc("Genuine Parts Company 30-03 Review Ave LONG ISLAND CITY NY 11101")

osha_current_loc = osha_current.progress_apply (lambda row: get_osha_current_loc(row),axis=1).apply(pd.Series)

In [80]:
osha_current_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 2 columns):
0    561 non-null float64
1    561 non-null float64
dtypes: float64(2)
memory usage: 9.0 KB


In [81]:
# osha_current_loc[osha_current_loc.duplicated(keep=False)]
osha_current['lat'] = osha_current_loc[0]
osha_current['lng'] = osha_current_loc[1]
osha_current[:1]

,date,description,employer,fatcat,inspection,victim,lat,lng
0,2017-04-28,A worker died after being caught between two vehicles.,"PROWLER RECOVERY, INC., 8100 W. Broward Blvd. PLANTATION FL 33324",Fatality,1229035.015,Francisco Canjura,26.122348,-80.258951


In [82]:
osha_current['date']= pd.to_datetime(osha_current['date'])
osha_current = osha_current.set_index((osha_current.select_dtypes(include=[np.datetime64]).columns).tolist())
osha_current.sort_index(inplace=True)
osha_current[:3]
# ok next combine that with the other osha stuff

,description,employer,fatcat,inspection,victim,lat,lng
date,,,,,,,
2016-05-24,Worker struck and killed by falling tree.,"Raymond Trent dba Raymond Trent, Morganfield, KY 42437",Fatality,1186003.015,James Finwick,37.683378,-87.916691
2016-06-27,Worker died from hantavirus after being exposed to mouse droppings while cleaning cabins.,"Marzano Mountain Retreat, Torreon, NM 87061",Fatality,1154167,Shawnee Romero,34.687556,-106.351590
2016-07-19,Worker killed in fall through stair hole.,"Johni Semma DBA Johni Semma, West Bloomfield, MI 48322",Fatality,1165245.015,Derek Ketz,42.544013,-83.346878


In [108]:
# osha_current[-3:]
osha_current.to_csv('osha_current.csv')
# todo, standardize before dropping stuff

In [109]:
osha_current.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 573 entries, 2016-05-24 to NaT
Data columns (total 7 columns):
description    573 non-null object
employer       573 non-null object
fatcat         573 non-null object
inspection     551 non-null object
victim         537 non-null object
lat            561 non-null float64
lng            561 non-null float64
dtypes: float64(2), object(5)
memory usage: 35.8+ KB


### OSHA Part 2, Archive CSVs 2015 and Before

In [111]:
osha_archive_urls = [
"https://www.osha.gov/dep/fatcat/FatalitiesFY09.csv",
"https://www.osha.gov/dep/fatcat/FatalitiesFY10.csv",
"https://www.osha.gov/dep/fatcat/FatalitiesFY11.csv",
"https://www.osha.gov/dep/fatcat/FatalitiesFY12.csv",
"https://www.osha.gov/dep/fatcat/fy13_federal-state_summaries.csv",
"https://www.osha.gov/dep/fatcat/fy14_federal-state_summaries.csv",
"https://www.osha.gov/dep/fatcat/fy15_federal-state_summaries.csv"]

osha_archive = pd.DataFrame()
for url in tqdm(osha_archive_urls):
    df = pd.DataFrame.from_csv(url)
#     print(df.info())
    osha_archive = pd.concat([osha_archive, df])
#     osha_current = pd.concat([osha_current, df])
#     print(url)
# all_osha.info()

100%|██████████| 7/7 [00:05<00:00,  1.27it/s]


In [112]:
# if a row is totally blank, drop it
osha_archive.dropna(axis=1, how="all", inplace=True)
osha_archive[:1]

,Company,"Company, City, State, ZIP",Date of Incident,Fatality or Catastrophe,Inspection #,Preliminary Description of Incident,Summary Report Date,Unnamed: 4,Victim(s)
FY 2009,"L.B. Thompson Inc., Tampa, FL 33610",NaN,6/23/2009,NaN,NaN,"Victim was inside a building bundling material and became ill, possible heat stroke.",7/24/2009,NaN,NaN


In [113]:
# make date it's own thing, but some are 'FY 09', not actual dates, also there's already Date of Incident but many are blank
osha_archive.reset_index(level=0,  inplace=True)
osha_archive['date'] = osha_archive['index']
osha_archive.drop('index', axis=1, inplace=True)

In [117]:

# date of incident is better than date, it takes presidence 
osha_archive['date'] = osha_archive['Date of Incident'].fillna(osha_archive['date'])
# and then drop the other,
osha_archive.drop('Date of Incident', axis=1, inplace=True)
# and then rename this date, 

#TODO need to include this in na fill for company/place, it has addresses.
osha_archive['Company'].fillna( osha_archive['Unnamed: 4'], inplace=True)
osha_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 9 columns):
Company                                3618 non-null object
Company, City, State, ZIP              3734 non-null object
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Unnamed: 4                             1094 non-null object
Victim(s)                              520 non-null object
date                                   7352 non-null object
dtypes: object(9)
memory usage: 519.3+ KB


In [118]:
# fill place with either company or company city state field
# because it could be either
osha_archive['place'] = osha_archive['Company'].fillna(osha_archive['Company, City, State, ZIP'])
# drop ones we don't have place field for
osha_archive.info()
osha_archive = osha_archive.dropna(axis=0, subset=['place'])
osha_archive.info()

# osha_archive[osha_archive.isnull()['Date of Incident'] ]
# osha_archive.info()

# osha_archive['Date of Incident'] = osha_archive['Date of Incident'].fillna( osha_archive['index'] )


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 10 columns):
Company                                3618 non-null object
Company, City, State, ZIP              3734 non-null object
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Unnamed: 4                             1094 non-null object
Victim(s)                              520 non-null object
date                                   7352 non-null object
place                                  7352 non-null object
dtypes: object(10)
memory usage: 577.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7352 entries, 0 to 7380
Data columns (total 10 columns):
Company                                3618 non-null object
Company, City, State, ZIP              3734 non-null object
Fatality or Catastrophe  

In [119]:

def get_osha_archive_loc(row):
    return get_lat_lng(row['place'])    

osha_archive_loc = osha_archive.progress_apply (lambda row: get_osha_archive_loc(row),axis=1).apply(pd.Series)


p: 100%|██████████| 7352/7352 [40:35<00:00,  7.01it/s]  


In [120]:
# osha_archive_loc.apply(pd.Series)
osha_archive['lat']=osha_archive_loc[0]
osha_archive['lng']=osha_archive_loc[1]

# osha_archive['Date of Incident']= pd.to_datetime(osha_archive['Date of Incident'])
# osha_archive = osha_archive.set_index((osha_archive.select_dtypes(include=[np.datetime64]).columns).tolist())
# osha_archive.sort_index(inplace=True)
# untried, Weds XXX
# osha_archive.index.rename('date', inplace=True)



In [123]:
osha_archive['Preliminary Description of Incident'].fillna('', inplace=True)
osha_archive['Company'].fillna('', inplace=True)
osha_archive.info()
# same for Company, and apply it, so no error below


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7352 entries, 0 to 7380
Data columns (total 12 columns):
Company                                7352 non-null object
Company, City, State, ZIP              3734 non-null object
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Unnamed: 4                             1094 non-null object
Victim(s)                              520 non-null object
date                                   7352 non-null object
place                                  7352 non-null object
lat                                    1221 non-null float64
lng                                    1221 non-null float64
dtypes: float64(2), object(10)
memory usage: 746.7+ KB


In [125]:
# just change name to description
osha_archive['description'] = osha_archive['Preliminary Description of Incident']

osha_archive[:5]    

,Company,"Company, City, State, ZIP",Fatality or Catastrophe,Inspection #,Preliminary Description of Incident,Summary Report Date,Unnamed: 4,Victim(s),date,place,lat,lng,description
0,"L.B. Thompson Inc., Tampa, FL 33610",NaN,NaN,NaN,"Victim was inside a building bundling material and became ill, possible heat stroke.",7/24/2009,NaN,NaN,6/23/2009,"L.B. Thompson Inc., Tampa, FL 33610",27.994928,-82.400987,"Victim was inside a building bundling material and became ill, possible heat stroke."
1,"Presiliana Zamora, Conroe, TX 77304",NaN,NaN,NaN,Victim was on a scaffold doing repairs on an archway and the scaffold collapsed.,7/24/2009,NaN,NaN,6/30/2009,"Presiliana Zamora, Conroe, TX 77304",NaN,NaN,Victim was on a scaffold doing repairs on an archway and the scaffold collapsed.
2,"Cesar Tapia, Miami, FL 33166",NaN,NaN,NaN,"Victim was conducting roofing work, and fell through a hole in the roof, 25 feet to the concrete floor below.",7/24/2009,NaN,NaN,7/6/2009,"Cesar Tapia, Miami, FL 33166",NaN,NaN,"Victim was conducting roofing work, and fell through a hole in the roof, 25 feet to the concrete floor below."
3,"Knobel Refrigeration, Inc., North Platte, NE 69103",NaN,NaN,NaN,Victim was climbing a ladder while servicing an air-conditioning unit and fell from the ladder.,7/24/2009,NaN,NaN,7/9/2009,"Knobel Refrigeration, Inc., North Platte, NE 69103",41.142954,-100.790990,Victim was climbing a ladder while servicing an air-conditioning unit and fell from the ladder.
4,"Greensburg, PA 15601",NaN,NaN,NaN,"Employees were throwing away old files. They had a dumpster on the forks in the raised position. In bringing the dumpster down on the forks, the load became unstable. Victim stood on the rear of the forklift to act as a counterbalance. The dumpster fell off the trucs, causing forktruck to lunge forward and the victim to strike his head on the overhead guard, falling to the ground.",7/24/2009,NaN,NaN,7/11/2009,"Greensburg, PA 15601",40.301458,-79.538929,"Employees were throwing away old files. They had a dumpster on the forks in the raised position. In bringing the dumpster down on the forks, the load became unstable. Victim stood on the rear of the forklift to act as a counterbalance. The dumpster fell off the trucs, causing forktruck to lunge forward and the victim to strike his head on the overhead guard, falling to the ground."


In [126]:
osha_archive.columns

Index(['Company', 'Company, City, State, ZIP', 'Fatality or Catastrophe',
       'Inspection #', 'Preliminary Description of Incident',
       'Summary Report Date', 'Unnamed: 4', 'Victim(s)', 'date', 'place',
       'lat', 'lng', 'description'],
      dtype='object')

In [127]:
# keep some,  'Company', 'place'
osha_archive.drop(['Company, City, State, ZIP',
       'Fatality or Catastrophe', 'Inspection #',
       'Preliminary Description of Incident', 'Summary Report Date',
       'Unnamed: 4', 'Victim(s)'], axis=1, inplace=True)
osha_archive[:4]    

,Company,date,place,lat,lng,description
0,"L.B. Thompson Inc., Tampa, FL 33610",6/23/2009,"L.B. Thompson Inc., Tampa, FL 33610",27.994928,-82.400987,"Victim was inside a building bundling material and became ill, possible heat stroke."
1,"Presiliana Zamora, Conroe, TX 77304",6/30/2009,"Presiliana Zamora, Conroe, TX 77304",NaN,NaN,Victim was on a scaffold doing repairs on an archway and the scaffold collapsed.
2,"Cesar Tapia, Miami, FL 33166",7/6/2009,"Cesar Tapia, Miami, FL 33166",NaN,NaN,"Victim was conducting roofing work, and fell through a hole in the roof, 25 feet to the concrete floor below."
3,"Knobel Refrigeration, Inc., North Platte, NE 69103",7/9/2009,"Knobel Refrigeration, Inc., North Platte, NE 69103",41.142954,-100.790990,Victim was climbing a ladder while servicing an air-conditioning unit and fell from the ladder.


In [132]:
get_lat_lng("arlington, va")
# osha_archive[]

# osha_archive.columns = ['date', 'lat', 'lng']

[nan, nan]

In [ ]:

# osha_archive['kind'] = 'w'
osha_archive.dropna(how='any', inplace=True)
osha_archive.info()

In [ ]:
# combine all the osha records, before splitting by time below
osha_all = pd.concat([osha_archive, osha_current])
# there are a couple with problems
# ?
# osha_all.dropna(axis=0, how='any', inplace=True)

In [ ]:
osha_all.info()
osha_all[:3]
# for the end result
# dfs_2015_2016.append(osha_all['2015':'2016'])
# dfs_all.append(osha_all)

### Combine Everything - REDO


In [ ]:
# combine them!
# deaths_all_years = pd.concat([vehicle_deaths, crime_deaths, ny_by_police, osha_all], ignore_index=True)
recent_deaths = pd.concat(dfs_2015_2016)
# recent_deaths.info()
# print('z')
# don't drop if we lack description

recent_deaths.dropna(axis=0, inplace=True, subset=['kind', 'lat', 'lng'])

In [ ]:
all_deaths = pd.concat(dfs_all)
# recent_deaths.info()
# print('z')
# don't drop if we lack description
all_deaths.dropna(axis=0, inplace=True, subset=['kind', 'lat', 'lng'])

In [ ]:
recent_deaths.sort_index(axis=0, inplace=True)
all_deaths.sort_index(axis=0, inplace=True)

In [ ]:
all_deaths.info() #.isnull().sum()

In [ ]:
recent_deaths.to_csv('recent_deaths.csv')

In [ ]:
all_deaths.to_csv('all_deaths.csv')

In [ ]:
# deaths.columns = ['d', 'k', 'lat', 'lng' ]
# deaths.to_json("deaths.json", orient="records")

In [ ]:
# deaths.info()

In [ ]:
# deaths.lat.value_counts()